In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
test=pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
import seaborn as sns
sns.lmplot(data=train, x="LotArea",y="SalePrice", hue="YrSold",fit_reg=False)

In [ ]:
#説明変数を以下のようにする
features=["MSZoning","LotArea","Neighborhood","ExterQual","CentralAir"]

In [ ]:

Xtrain=train[features]
ytrain=train["SalePrice"]
Xtest=test[features]


Xtrain.head()

In [ ]:
Xtrain=pd.get_dummies(Xtrain)
Xtest=pd.get_dummies(Xtest)

In [ ]:
print(Xtrain.shape)

print(ytrain.shape)
print(Xtest.shape)

Xtrain.head()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#パラメータ調節
def param():
    param_grid= {
        'alpha': [0.001, 0.01, 0.1, 1, 10],
        'l1_ratio': [0,0.1, 0.25, 0.5, 0.75]}
    return param_grid


grid_search = GridSearchCV(ElasticNet(max_iter=100000), param(), cv=StratifiedKFold(n_splits=3), n_jobs=-1)#CPUの力をmax
grid_search.fit(Xtrain, ytrain)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

In [ ]:
grid_scores = pd.DataFrame(grid_search.cv_results_)
scores = np.array(grid_scores.mean_test_score).reshape(5,5)#ハイパーパラメータは5個×5個
print(scores)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
mat = ax.matshow(scores, cmap='Reds', alpha=0.8)
fig.colorbar(mat)
ax.set_xticks(range(5))
ax.set_xticklabels([0.001, 0.01, 0.1, 1, 10])
ax.set_xlabel('alpha')
ax.set_yticks(range(5))
ax.set_yticklabels([0,0.1, 0.25, 0.5, 0.75])
ax.set_ylabel('l1 ratio')

In [ ]:
clf=ElasticNet(alpha=0.001, l1_ratio=0.25,max_iter=100000)
clf.fit(Xtrain,ytrain)
predictions=clf.predict(Xtest)


In [ ]:
output = pd.DataFrame(predictions,columns=["SalePrice"])#Idはsample_submissionの形式をコピペしました。

output.to_csv('my_submission.csv')
print(output.head())

print("my submission was successfully saved!")